In [1]:
import os
import requests
import torch
import numpy as np
import torch.optim.adamw
from MLA import Model

input_file_path = 'input.txt'
if not os.path.exists(input_file_path):
    data_url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
    with open(input_file_path, 'w', encoding='utf-8') as f:
        f.write(requests.get(data_url).text)

with open(input_file_path, "r", encoding="utf-8") as f:
    data = f.read()

print(len(data))
##data = "data"
vocab = sorted(list(set(data)))
vocab_size = len(vocab)

print(vocab.index('t'))

tokens = [vocab.index(v) for i, v in enumerate(data)]

print(tokens[:40])
print(vocab)

maxSteps = 3000
gradAccum = 1
batchSize = 16
n_head = 12
n_layers = 12
n_embd = n_head * 32
blocksize = 1024

#MHA
#model = Model(n_layers,n_embd,n_head,vocab_size,blocksize).to("cuda")

model = Model(n_layers,n_embd,n_head,vocab_size,blocksize, LCompression=288, flashATTN=True).to("cuda")
#model = torch.compile(model)
tot = 0
for param in model.parameters():
    tot += param.numel()

print(f"total params: {tot//1e6}M params")


optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
def get_batch():
    ix = torch.randint(len(tokens) - blocksize, (batchSize,))
    x = torch.stack([torch.tensor(tokens[i:i+blocksize], dtype=torch.long) for i in ix])
    y = torch.stack([torch.tensor(tokens[i+1:i+blocksize+1], dtype=torch.long) for i in ix])
    return x.to("cuda"), y.to("cuda")

import time
for i in range(maxSteps):
    xo = time.time()
    optimizer.zero_grad()
    for _ in range(gradAccum):
        x, y = get_batch()
        out, loss = model(x, y)
        loss.backward()
    optimizer.step()
    if i % 10 == 0:
        t1 = time.time() - xo
        print(f"step: {i}/{maxSteps}, loss: {loss.item():.6f}, t/step: {t1:.4f}s, time left: {t1 * (maxSteps-i):.2f}")


generated = torch.tensor([0], device="cuda").unsqueeze(0)
from torch import nn
for i in range(200):
    out = model(generated)
    out = out[:,-1,:]
    
    out = nn.functional.softmax(out, dim=-1)
    predchar = torch.multinomial(out, num_samples=1)
    
    
    generated = torch.concat([generated, predchar], dim=-1)


outText = [vocab[int(v.item())] for i, v in enumerate(generated.squeeze(0))]
print("".join(outText))

1115394
58
[18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10, 0, 14, 43, 44, 53, 56, 43, 1, 61, 43, 1, 54, 56, 53, 41, 43, 43, 42, 1, 39, 52, 63, 1, 44, 59, 56]
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
total params: 47.0M params
step: 0/3000, loss: 4.349799, t/step: 1.0553s, time left: 3166.02
step: 10/3000, loss: 2.917651, t/step: 0.7197s, time left: 2152.03


KeyboardInterrupt: 